## NoteBook for the first project
- Je pensais que l'on pouvait essayer de faire ca comme ceci:
    - Chacun de nous a un block (peut bien entendu en ajouter en dessous du sien) afin de ne pas avoir trop de conflit lorsque l'on git push
        - A tester cette semaine pour voir si c'est ok

# Stefan's Blocks

In [ ]:
# First block for Stefan

In [1]:
print('Hello WORLD!!!! By Stef')

Hello WORLD!!!! By Stef


# Etienne's Blocks

In [ ]:
# First block for Etienne
print('My first contribution TEST')

# Ivan's Blocks

In [1]:
# First block for Ivan
print('Salut kikoo')

Salut kikoo


In [2]:
import numpy as np

In [1]:
print('encore une modification')

encore une modification
